# Motivation
Now, for a very different type of notebook: I'm leaving the realm of computer vision, and moving into some algorithm design. In this notebook, I'll write an algorithm to solve a Boggle board. 

# Setup
The cells below will set up the rest of the notebook. 

I'll start by configuring the kernel:

In [ ]:
# Change directories to the root of the project
%cd ..

# Enable autoreload of modules
%load_ext autoreload
%autoreload 2

Next, I'm going to import various modules:

In [ ]:
# Import statements
import json
from pathlib import Path
import pandas as pd
from tqdm import tqdm

# Import custom-built modules 
from utils.settings import allowed_boggle_tiles

I'm also going to load in the dictionary of allowed words:

In [ ]:
# Load in the list of allowed words
with open("data\scrabble-dictionary.json", "r") as json_file:
    allowed_words = json.load(json_file)

# Methods
Below, I'll define a number of methods. 

This first one will build [a trie](https://en.wikipedia.org/wiki/Trie). This is an efficient data structure for string search, apparently.  

In [ ]:
def build_trie(words):
    """
    This method will build a trie from a list of words.
    """
    
    trie = {}
    for word in words:
        node = trie
        for char in word:
            if char not in node:
                node[char] = {}
            node = node[char]
        node["end"] = True
    return trie

def build_trie(words, special_tiles):
    trie = {}
    for word in words:
        if word=="minister":
            print("HEY")
        node = trie
        i = 0
        while i < len(word):
            char = word[i]
            if word=="minister":
                print(char)
            # Check if the character has a special mapping
            special_tile = special_tiles.get(char, None)
            if special_tile and word.startswith(special_tile, i):
                char = special_tile
                i += len(special_tile)
            else:
                i += 1
            if char not in node:
                node[char] = {}
            node = node[char]
        node["end"] = True
    return trie

def build_trie(words, special_tiles):
    trie = {}
    for word in words:
        node = trie
        i = 0
        while i < len(word):
            char = word[i]
            # Check if the character has a special mapping
            special_tile = special_tiles.get(char, None)
            
            # Add the special tile to the trie
            if special_tile and word.startswith(special_tile, i):
                if special_tile not in node:
                    node[special_tile] = {}
                temp_node = node[special_tile]
                i += len(special_tile)
                for next_char in word[i:]:
                    if next_char not in temp_node:
                        temp_node[next_char] = {}
                    temp_node = temp_node[next_char]
                temp_node["end"] = True
            
            # Add the individual letter to the trie
            if char not in node:
                node[char] = {}
            node = node[char]
            i += 1

        node["end"] = True
    return trie

def add_word_to_trie(trie, word, i, special_tiles):
    if i == len(word):
        trie["end"] = True
        return

    char = word[i]
    special_tile = special_tiles.get(char, None)

    # Handle special tile
    if special_tile and word.startswith(special_tile, i):
        if special_tile not in trie:
            trie[special_tile] = {}
        add_word_to_trie(trie[special_tile], word, i + len(special_tile), special_tiles)

    # Handle individual letter
    if char not in trie:
        trie[char] = {}
    add_word_to_trie(trie[char], word, i + 1, special_tiles)

def build_trie(words, special_tiles):
    trie = {}
    for word in words:
        add_word_to_trie(trie, word, 0, special_tiles)
    return trie

Next method will parse a Boggle board from a semicolon-delimited letter sequence:

In [ ]:
def parse_board_from_letter_sequence(letter_sequence):
    """
    This method will parse a Boggle board (in the form of a 2D list)
    from a sequence of semi-colon delimited letters.
    """
    board = []
    split_letter_sequence = letter_sequence.split(";")
    n_rows = int(len(split_letter_sequence) ** 0.5)
    for i in range(n_rows):
        board.append(split_letter_sequence[i*n_rows:(i+1)*n_rows])
    return board

The next method will score a word depending on its length.

In [ ]:
def score_boggle_word(word):
    """
    This method will score a Boggle word.
    """

    if len(word) < 4:
        return 0
    elif len(word) == 4:
        return 1
    elif len(word) == 5:
        return 2
    elif len(word) == 6:
        return 3
    elif len(word) == 7:
        return 5
    else:
        return 11

These next couple of methods were generated by ChatGPT - they'll help me solve the Boggle boards:

In [ ]:
# Check if a position is within the Boggle board
def is_valid(x, y, visited, board):
    return 0 <= x < len(board) and 0 <= y < len(board[0]) and not visited[x][y]


# Run DFS on the board to find words
def dfs(x, y, board, visited, trie, word, found_words):
    if "end" in trie:
        found_words.add(word)

    visited[x][y] = True

    # Adjacent positions (up, down, left, right, and diagonals)
    directions = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]

    for dx, dy in directions:
        new_x, new_y = x + dx, y + dy
        if is_valid(new_x, new_y, visited, board) and board[new_x][new_y] != "block":
            next_char = board[new_x][new_y]
            if len(next_char) == 0:
                if next_char in trie:
                    dfs(
                        new_x,
                        new_y,
                        board,
                        visited,
                        trie[next_char],
                        word + next_char,
                        found_words,
                    )
            else:
                for char in next_char:
                    if char in trie:
                        dfs(
                            new_x,
                            new_y,
                            board,
                            visited,
                            trie[char],
                            word + char,
                            found_words,
                        )

    visited[x][y] = False


# Solve the Boggle board
def solve_boggle(board, trie):
    found_words = set()

    rows, cols = len(board), len(board[0])
    visited = [[False for _ in range(cols)] for _ in range(rows)]

    for i in range(rows):
        for j in range(cols):
            start_char = board[i][j]
            if start_char in trie:
                dfs(i, j, board, visited, trie[start_char], start_char, found_words)
                
    # Now, we're going to create a DataFrame that contains the words found on each board
    available_words_df = pd.DataFrame.from_records([
        {
            "word": word,
            "length": len(word),
            "points": score_boggle_word(word)
        } for word in found_words
    ]).sort_values("length", ascending=False)

    return available_words_df

# Main Loop
Below, I'm going to load in all of the Boggle boards, and then solve them. I'll start by loading them in.

In [ ]:
# Open the .csv file containing the labeled boards
board_data_df = pd.read_csv("data/labeled-boards.csv")

# Convert all of the letters to lowercase
board_data_df["letter_sequence"] = board_data_df["letter_sequence"].str.lower()

# Parse the boards from the letter sequences
board_data_df["board"] = board_data_df["letter_sequence"].apply(parse_board_from_letter_sequence)

Now that I've got each of the boards, I can run board detection on them. 

In [ ]:
# Create a trie out of the allowed words
allowed_words_trie = build_trie(allowed_words, {tile[0].lower():tile.lower() for tile in allowed_boggle_tiles if len(tile) > 1})

# Now, create a dictionary mapping image names to DataFrames containing the 
# allowed words and their scores
board_data_dfs = {}
for row in tqdm(list(board_data_df.itertuples())):
    board_data_dfs[row.file_path] = solve_boggle(row.board, allowed_words_trie)

Now, I'm going to calculate some stats about each board, and add them to a DataFrame.

In [ ]:
# This list will hold DataFrame records
solved_board_stats_df_records = []

# Iterate through each of the boards
for board_file_path, solved_board_df in board_data_dfs.items():
    
    # Determine how many words are worth 11 points
    max_point_word_ct = len(solved_board_df.query("points==11"))
    
    # Determine how many points are available on the board
    max_points = solved_board_df["points"].sum()
    
    # Store the stats in a dictionary
    solved_board_stats_df_records.append({
        "file_path": board_file_path,
        "max_point_word_ct": max_point_word_ct,
        "max_points": max_points
    })

# Create a DataFrame from the records
solved_board_stats_df = pd.DataFrame.from_records(solved_board_stats_df_records)

# Merge the solved board stats with the original board data
solved_board_stats_df = board_data_df.merge(solved_board_stats_df, on="file_path").sort_values(
    "max_points", ascending=False
)

In [ ]:
solved_board_stats_df["max_point_word_ct"].mean()

In [ ]:
solved_board_stats_df.query("file_path=='data\\test-pictures\\easy-24.png'")

In [ ]:
board_data_dfs["data\\test-pictures\\easy-24.png"].head(15)